In [1]:
import numpy as np
import xarray as xr
import metpy.calc
from metpy.calc import cape_cin, dewpoint_from_relative_humidity, parcel_profile, showalter_index,equivalent_potential_temperature,get_layer,moist_lapse
from metpy.units import units
from wrf import interpz3d,getvar
from numpy import sqrt,mod,arctan2
import os
import pandas as pd
import shutil
import xlrd
import xlwt
import matplotlib 
import proplot as pplt
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from netCDF4 import Dataset
from datetime import datetime

/home/hpp/anaconda3/envs/conv/lib/python3.8/site-packages/proplot/__init__.py:46: ProPlotWarning: Ignoring the following .ttc fonts because they cannot be saved into PDF or EPS files (see matplotlib issue #3135): '/home/hpp/.config/proplot/fonts/Sitka.ttc', '/home/hpp/.config/proplot/fonts/SitkaB.ttc', '/home/hpp/.config/proplot/fonts/SitkaI.ttc', '/home/hpp/.config/proplot/fonts/SitkaZ.ttc', '/home/hpp/.config/proplot/fonts/YuGothB.ttc', '/home/hpp/.config/proplot/fonts/YuGothL.ttc', '/home/hpp/.config/proplot/fonts/YuGothM.ttc', '/home/hpp/.config/proplot/fonts/YuGothR.ttc', '/home/hpp/.config/proplot/fonts/cambria.ttc', '/home/hpp/.config/proplot/fonts/mingliub.ttc', '/home/hpp/.config/proplot/fonts/msgothic.ttc', '/home/hpp/.config/proplot/fonts/msjh.ttc', '/home/hpp/.config/proplot/fonts/msjhbd.ttc', '/home/hpp/.config/proplot/fonts/msjhl.ttc', '/home/hpp/.config/proplot/fonts/msyh.ttc', '/home/hpp/.config/proplot/fonts/msyhbd.ttc', '/home/hpp/.config/proplot/fonts/msyhl.ttc', '/h

### 读文件

In [106]:
# file_high='/mnt/d/hpp_onedrive/OneDrive/work/data/era5/20210415.nc'
# file_surf='/mnt/d/hpp_onedrive/OneDrive/work/data/era5/20210415-sfc.nc'
file_high='/mnt/d/OneDrive/work/data/era5/20210415.nc'
file_surf='/mnt/d/OneDrive/work/data/era5/20210415-sfc.nc'

##3333333333333333333333世界时
time_star=7
time_end =22
time= pd.date_range('2021-04-15-07','2021-04-15-21',freq='H')
###33333333333333333333333北京时
day='2021_041515-1605'
# label=['15-15','15-16','15-17','15-18','15-19','15-20','15-21','15-22','15-23','16-00','16-01','16-02','16-03','16-04','16-05']#,'29-02','29-03']#,'16-01','16-02','16-03','16-04','16-05']

era_high=xr.open_dataset(file_high)
geo_time=era_high['z'][time_star:time_end,::-1,::-1,:]##################################################
z_time=metpy.calc.geopotential_to_height(geo_time)
p_time=era_high['z'][time_star:time_end,:,:,:]['level'].values[::-1]* units.hPa
r_time=era_high['r'][time_star:time_end,::-1,::-1,:]*0.01
t_time=era_high['t'][time_star:time_end,::-1,::-1,:]-273.15
u_time=era_high['u'][time_star:time_end,::-1,::-1,:]
v_time=era_high['v'][time_star:time_end,::-1,::-1,:]
w_time=era_high['w'][time_star:time_end,::-1,::-1,:]
era_surf=xr.open_dataset(file_surf)
u10_time=era_surf['u10'][time_star:time_end,::-1,:]
v10_time=era_surf['v10'][time_star:time_end,::-1,:]

dur=time_end-time_star
var=['CAPE','LI','DCAPE','SWEAT','800-500','0-3KM','0-6KM']

### 读连续文件

In [ ]:
file_high_0='/mnt/d/hpp_onedrive/OneDrive/work/era5/20200809.nc'
file_surf_0='/mnt/d/hpp_onedrive/OneDrive/work/era5/20200809-sfc.nc'
file_surf_1='/mnt/d/hpp_onedrive/OneDrive/work/era5/20200810-sfc.nc'
file_high_1='/mnt/d/hpp_onedrive/OneDrive/work/era5/20200810.nc'

era_high_0=xr.open_dataset(file_high_0)
era_high_1=xr.open_dataset(file_high_1)
era_surf_0=xr.open_dataset(file_surf_0)
era_surf_1=xr.open_dataset(file_surf_1)
era_high=xr.concat([era_high_0,era_high_1],dim="time")
era_surf=xr.concat([era_surf_0,era_surf_1],dim="time")

#世界时
time_star=16
time_end =25
###北京时
day='20_081000-1008'
label=['10-00','10-01','10-02','10-03','10-04','10-05','10-06','10-07','10-08']#,'05-02','05-03','05-04','05-05','05-06','05-07','05-08','05-09','05-10','05-11']

geo_time=era_high['z'][time_star:time_end,::-1,::-1,:]##################################################
z_time=metpy.calc.geopotential_to_height(geo_time)
p_time=era_high['z'][time_star:time_end,:,:,:]['level'].values[::-1]* units.hPa
r_time=era_high['r'][time_star:time_end,::-1,::-1,:]*0.01
t_time=era_high['t'][time_star:time_end,::-1,::-1,:]-273.15
u_time=era_high['u'][time_star:time_end,::-1,::-1,:]
v_time=era_high['v'][time_star:time_end,::-1,::-1,:]
w_time=era_high['w'][time_star:time_end,::-1,::-1,:]

u10_time=era_surf['u10'][time_star:time_end,::-1,:]
v10_time=era_surf['v10'][time_star:time_end,::-1,:]

dur=time_end-time_star
time = np.arange(dur)


### dcape

In [3]:
def downdraft_cape(pressure, temperature, dewpoint, parcel=None, bottom=None,
                    depth=400 * units.hPa):
    r"""Calculate downdraft CAPE.
    Calculate the downdraft convective available potential energy (CAPE).The minimum theta-e
    value between the surface and top_pressure is used as the parcel starting point. Parcels
    descend along a moist adiabat. Area between the parcel path and environmental temperature
    is integrated to calculate DCAPE.
    Parameters
    ----------
    pressure : `pint.Quantity`
        The atmospheric pressure level(s) of interest. The first entry should be the starting
        point pressure.
    temperature : `pint.Quantity`
        The atmospheric temperature corresponding to pressure.
    dewpoint : `pint.Quantity`
        The atmospheric dew point corresponding to pressure.
    parcel : tuple
        Tuple of pressure and temperature for the starting parcel.
    bottom : `pint.Quantity`
        Lowest point in the parcel path to consider in integration in height or pressure.
        If no heights are given, a standard atmosphere will be assumed. Defaults to None.
    top_pressure : `pint.Quantity`
        The lowest pressure value to be considered in the calculation. Defaults to 400 hPa.
    Returns
    -------
    `pint.Quantity`
        Downdraft convective available potential energy (CAPE).
    Notes
    -----
    .. math:: \text{DCAPE} = R_d \int_{P_i}^{P_{sfc}} (T_{env} - T_{parcel}) d\text{ln}(p)
    * :math:`DCAPE` Downdraft convective available potential energy
    * :math:`R_d` Gas constant
    * :math:`g` Gravitational acceleration
    * :math:`T_{parcel}` Parcel temperature
    * :math:`T_{env}` Environment temperature
    * :math:`p` Atmospheric pressure
    """

    # If the user specified a parcel starting point, we'll use that instead of the default
    if parcel:
        if depth:
            ValueError('Cannot specify a depth when using a custom parcel.')

        parcel_starting_pressure, parcel_starting_temperature = parcel

        # If no bottom is specified, we'll use the surface (default of get_layer),but
        # done explicitly here to depth calculation.
        if not bottom:
            bottom = np.nanmax(pressure) * pressure.units

        # Calculate the depth of the sounding to use (bottom - the parcel starting point)
        depth = bottom - parcel_starting_pressure

        # Trim data to the bottom and depth above it
        pressure, temperature = get_layer(pressure, temperature, bottom=bottom,
                                          depth=depth)

    # # The user did not give us a parcel, so we'll calculate a sensible default.

    else:

        # Trim data to only top height and below as well as bottom height and above
        pressure, temperature, dewpoint = get_layer(pressure, temperature, dewpoint,
                                                    depth=depth,
                                                    bottom=bottom
                                                    )

        # Find minimum theta-e
        theta_e = equivalent_potential_temperature(pressure, temperature, dewpoint)
        minimum_theta_e_index = np.argmin(theta_e)

        # Trim data to be minimum theta-e down (dewpoint is no longer required)
        if len(pressure[:minimum_theta_e_index]) != 0:
            
            pressure = pressure[:minimum_theta_e_index]
            temperature = temperature[:minimum_theta_e_index]

            # Sort for monotonically increasing pressure that trapz needs to work.
            # Also guarantees the logic for calculating a descending parcel is sound.
            sort_inds = np.argsort(pressure)
            pressure = pressure[sort_inds]
            temperature = temperature[sort_inds]

            # Create the parcel profile for decent along a moist adiabat
            profile_temperatures = moist_lapse(pressure, temperature[0])

            # Calculate the difference in profile and environmental temperature to integrate

            y_vals = temperature - profile_temperatures

            # Calculate DCAPE
            dcape = metpy.constants.dry_air_gas_constant * (np.trapz(y_vals,
                                x=np.log(pressure.m)) )
        else:
            dcape=np.nan*units('J/kg')
    return dcape.to('J/kg')#, pressure[::-1], profile_temperatures[::-1]

### 区域

In [43]:
# var=np.zeros((dur,var_num))
var_mean=pd.DataFrame(columns=['CAPE', 'LI', 'DCAPE','SWEAT','800-500','0-3KM','0-6KM'], index=time)
for i in range(0,dur):
    #CAPE   
    t_mean=((t_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5]) )* units.degC
    r_mean=(r_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5])* units.dimensionless
    Td = dewpoint_from_relative_humidity(t_mean, r_mean)
    prof= parcel_profile(p_time, np.mean(t_mean,axis=(1,2))[0], np.mean(Td,axis=(1,2))[0]).to('degC')
    var_mean['CAPE'][i]=cape_cin(p_time, np.mean(t_mean,axis=(1,2)), np.mean(Td,axis=(1,2)), prof)[0].magnitude #cape
    # #LI
    var_mean['LI'][i]=metpy.calc.lifted_index(p_time, np.mean(t_mean,axis=(1,2)), prof, vertical_dim=0)[0].magnitude #li
    # #DCAPE
    var_mean['DCAPE'][i]=downdraft_cape(p_time, np.mean(t_mean,axis=(1,2)), np.mean(Td,axis=(1,2))).magnitude
    #sweat
    wspd=sqrt( (u_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5])*(u_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5])+
            (v_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5])*(v_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5]) )*units.knot
#     wdir=mod(180.0+arctan2(u_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5],v_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5]),360.0)
    wdir=metpy.calc.wind_direction(u_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5], v_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5], convention='from')
    var_mean['SWEAT'][i]=metpy.calc.sweat_index(p_time,np.mean(t_mean,axis=(1,2)),np.mean(Td,axis=(1,2)),wspd.mean(),wdir.mean(),vertical_dim=0)[0].magnitude  #sweat
    #xiangdangweiwen
    var_mean['800-500'][i] = equivalent_potential_temperature(p_time, np.mean(t_mean,axis=(1,2)), np.mean(Td,axis=(1,2)) )[6].values-equivalent_potential_temperature(p_time, np.mean(t_mean,axis=(1,2)), np.mean(Td,axis=(1,2)) )[15].values 
    #0-3km shear
    z_list=[6000,3000] #unit:m
    u_height = interpz3d(u_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5],z_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5],np.array(z_list))
    v_height = interpz3d(v_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5],z_time[i,:,:,:].loc[:,36.1:41.5,117.5:122.5],np.array(z_list))
    var_mean['0-3KM'][i]=( ((u_height[1,:,:]-u10_time[i,:,:].loc[36.1:41.5,117.5:122.5])**2+(v_height[1,:,:]-v10_time[i,:,:].loc[36.1:41.5,117.5:122.5])**2)**(1/2) ).mean().values #3km
    #0-6km shear
    var_mean['0-6KM'][i]=( ((u_height[0,:,:]-u10_time[i,:,:].loc[36.1:41.5,117.5:122.5])**2+(v_height[0,:,:]-v10_time[i,:,:].loc[36.1:41.5,117.5:122.5])**2)**(1/2) ).mean().values #6km


<ipython-input-43-84e26c7a99e8>:7: UserWarning: Relative humidity >120%, ensure proper units.
  Td = dewpoint_from_relative_humidity(t_mean, r_mean)
<ipython-input-43-84e26c7a99e8>:7: UserWarning: Relative humidity >120%, ensure proper units.
  Td = dewpoint_from_relative_humidity(t_mean, r_mean)
<ipython-input-43-84e26c7a99e8>:7: UserWarning: Relative humidity >120%, ensure proper units.
  Td = dewpoint_from_relative_humidity(t_mean, r_mean)
/home/hpp/anaconda3/envs/conv/lib/python3.8/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)


### 站点

In [5]:
sta=['54539','54662','54778',
'D3118','D2002','54741','54538','54721','54449','54720','54647','54641','54544',
'L5070','L2245','L5069','L2215','L2239','L6081',
'A3171',
'54548','54642','54646','54639','54743','54649','54654','54552','54560','54464','54465']

sta_lon=[118.89,121.64,122.48,
119.1883,119.1914,118.8125,118.4544,117.8836,119.5103,117.7089,118.8742,118.5483,119.2792,
121.47,121.8769,121.0553,121.5136,121.3819,122.0742,
117.7892,
119.3583,119.3325,118.4131,119.0656,119.5492,119.8331,120.1508,120.6117,121.0583,121.1044,121.4128]

sta_lat=[39.43,38.91,37.17,
37.7589,37.2511,38.2153,39.0836,38.3431,39.8469,38.4575,39.1681,38.855,39.5683,
40.94,39.9833,40.7933,39.7797,39.4153,40.2158,
38.9781,
39.155,38.9514,38.4433,38.5503,38.1175,38.0831,38.3192,39.7983,39.9775,40.3397,40.6639]

In [6]:
# 定义一个方法
def Bilinear_interp(lonSta, latSta, longitude, latitude, var):
    var_sta = []
    for i in range(len(lonSta)):
        iSta = np.searchsorted(longitude, lonSta[i]) 
        if longitude[iSta] > lonSta[i]:
            iSta = iSta - 1 # 经度左下角点在经度array里的索引
            
        jSta = np.searchsorted(latitude, latSta[i])
        if latitude[jSta] > latSta[i]:
            jSta = jSta - 1 # 纬度左下角点在纬度array里的索引
        
        var11 = var[jSta, iSta]      #### t2m -> var
        var21 = var[jSta, iSta+1]
        var12 = var[jSta+1, iSta]
        var22 = var[jSta+1, iSta+1]

		# 改变变量名，为了公式计算时候方便
        x = lonSta[i]
        y = latSta[i]
        x1 = longitude[iSta]
        x2 = longitude[iSta+1]
        y1 = latitude[jSta]
        y2 = latitude[jSta+1]
        arg   = 1.0 / ((x2 - x1)*(y2 - y1))
        arg11 = arg * (x2 - x) * (y2 - y) 
        arg21 = arg * (x - x1) * (y2 - y)
        arg12 = arg * (x2 - x) * (y - y1)
        arg22 = arg * (x - x1) * (y - y1)
        var_interp = arg11*var11  + arg21*var21 + arg12*var12 + arg22*var22
        var_sta.append(var_interp)
    return var_sta

### 插值

In [7]:
var3_inter={}
var3_list=['t_sta','r_sta','u_sta','v_sta','wspd_sta','wdir_sta','z_sta']
for i in var3_list:
    var3_inter[i]=np.zeros((dur,len(sta),27))

var32_inter={}
var32_list=['uheight_sta','vheight_sta']
for i in var32_list:
    var32_inter[i]=np.zeros((dur,len(sta),2))

var2_inter={}
var2_list=['u10_sta','v10_sta']
for i in var2_list:
    var2_inter[i]=np.zeros((dur,len(sta)))

lat=r_time['latitude'].values
lon=r_time['longitude'].values
z_list=[6000,3000]
for i in range(0,dur):###########################################################
    wspd=sqrt( u_time[i,:,:,:]*u_time[i,:,:,:])+(v_time[i,:,:,:]*v_time[i,:,:,:] )
    wdir=metpy.calc.wind_direction(u_time[i,:,:,:], v_time[i,:,:,:], convention='from')
    u_height = interpz3d(u_time[i,:,:,:],z_time[i,:,:,:],np.array(z_list))
    v_height = interpz3d(v_time[i,:,:,:],z_time[i,:,:,:],np.array(z_list))
    for j in range(0,len(sta)):
        for k in range(0,27):
            var3_inter['t_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, t_time[i,k,:,:])[0]
            var3_inter['r_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, r_time[i,k,:,:])[0]
            var3_inter['u_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, u_time[i,k,:,:])[0]
            var3_inter['v_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, v_time[i,k,:,:])[0]
            var3_inter['z_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, z_time[i,k,:,:])[0]
            var3_inter['wspd_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, wspd[k,:,:])[0]
            var3_inter['wdir_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, wdir[k,:,:])[0]
        var2_inter['u10_sta'][i,j] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, u10_time[i,:,:])[0]
        var2_inter['v10_sta'][i,j] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, v10_time[i,:,:])[0]
        for k in range(0,2):
            var32_inter['uheight_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, u_height[k,:,:])[0]
            var32_inter['vheight_sta'][i,j,k] = Bilinear_interp([sta_lon[j]], [sta_lat[j]], lon, lat, v_height[k,:,:])[0]

In [21]:
sta_var={}
for j in range(0,len(sta)):
    sta_var[sta[j]]=pd.DataFrame(columns=['CAPE', 'LI', 'DCAPE','SWEAT','800-500','0-3KM','0-6KM'], index=time)
    for i in range(0,dur):
        t_mean=var3_inter['t_sta'][i,j,:]* units.degC
        r_mean=var3_inter['r_sta'][i,j,:]* units.dimensionless
        Td = dewpoint_from_relative_humidity(t_mean, r_mean)
        prof= parcel_profile(p_time, t_mean[0], Td[0]).to('degC')
        if (cape_cin(p_time, t_mean, Td, prof)[0].magnitude)>0:
            sta_var[sta[j]]['CAPE'][i]=cape_cin(p_time, t_mean, Td, prof)[0].magnitude 
        else:
            sta_var[sta[j]]['CAPE'][i]=0
        # #LI
        sta_var[sta[j]]['LI'][i]=metpy.calc.lifted_index(p_time, t_mean, prof, vertical_dim=0)[0].magnitude 
        # #DCAPE
        sta_var[sta[j]]['DCAPE'][i]=downdraft_cape(p_time, t_mean, Td).magnitude
        #sweat
        sta_var[sta[j]]['SWEAT'][i]=metpy.calc.sweat_index(p_time,t_mean,Td,var3_inter['wspd_sta'][i,j,:]*units("m/s"),var3_inter['wdir_sta'][i,j,:],vertical_dim=0)[0]
        #xiangdangweiwen
        sta_var[sta[j]]['800-500'][i] = equivalent_potential_temperature(p_time, t_mean, Td )[6].magnitude-equivalent_potential_temperature(p_time, t_mean, Td)[15].magnitude 
        #0-3km shear
        sta_var[sta[j]]['0-3KM'][i]= ((var32_inter['uheight_sta'][i,j,1]-var2_inter['u10_sta'][i,j])**2+(var32_inter['vheight_sta'][i,j,1]-var2_inter['v10_sta'][i,j])**2)**(1/2)
        #0-6km shear
        sta_var[sta[j]]['0-6KM'][i]= ((var32_inter['uheight_sta'][i,j,0]-var2_inter['u10_sta'][i,j])**2+(var32_inter['vheight_sta'][i,j,0]-var2_inter['v10_sta'][i,j])**2)**(1/2)

### picture

In [ ]:
path_output = "/mnt/d/hpp_OneDrive/OneDrive/work/output/渤海中西部对流参数/test/"+day+'/'
if not os.path.exists(path_output):
    os.makedirs(path_output)
    
matplotlib.rcParams['xtick.labelsize'] = 8
matplotlib.rcParams['ytick.labelsize'] = 8
matplotlib.rcParams["font.weight"] = "bold"
fontdict = {'fontsize': 12,'fontweight' : 800}
fig,axs=plt.subplots(ncols=3,nrows=3,figsize=(24,10),sharey = False)
axs[2,1].remove()
axs[2,2].remove()
axes = axs.flatten()
for i,element in enumerate(var):
    axes[i].plot(var_mean.iloc[:,i])
    axes[i].set_title(var[i],fontdict=fontdict)
    for x,y in zip(var_mean.index,var_mean.iloc[:,i]):
        axes[i].text(x,y,'%.1f' % y,ha='center', va= 'bottom',fontdict={'fontsize':7})
axs[2,0].set_xlabel('time(BJ)', fontsize =10,fontweight ='bold')
fig.suptitle('mean',fontsize=15)
plt.subplots_adjust(wspace=0.15,hspace=0.3)
plt.savefig(path_output+"区域.jpg",dpi=600, bbox_inches='tight')

for k in range(0,len(sta)-30):
    fig,axs=plt.subplots(ncols=3,nrows=3,figsize=(24,10),sharey = False,sharex = False)
    axs[2,1].remove()
    axs[2,2].remove()
    axes = axs.flatten()
    for i,element in enumerate(var):
        axes[i].plot(sta_var[sta[k]].iloc[:,i])
        for x,y in zip(sta_var[sta[k]].index,sta_var[sta[k]].iloc[:,i]):
            axes[i].text(x,y,'%.1f' % y,ha='center', va= 'bottom',fontdict={'fontsize':7})
    fig.suptitle(sta[k],fontsize=15)
    plt.subplots_adjust(wspace=0.15,hspace=0.3)
    plt.savefig(path_output+sta[k]+".jpg",dpi=600, bbox_inches='tight')

### 写文件

In [107]:
# path_output = "/mnt/d/hpp_onedrive/OneDrive/work/output/渤海中西部对流参数/test/"
path_output = "/mnt/d/OneDrive/work/output/渤海中西部对流参数/test/"
sheet={}
sheet_name=['sheet1','sheet2','sheet3','sheet4','sheet5','sheet6','sheet7']
# path_output = "/mnt/d/OneDrive/work/code/渤海中西部对流参数_excel/"
if not os.path.exists(path_output):
    os.makedirs(path_output)
f = xlwt.Workbook()
for i,name in enumerate(sheet_name):
    sheet[name]=f.add_sheet(var[i],cell_overwrite_ok=True) 
    sheet[name].write(0,1,'mean')
    for j in range(0,len(time)):
        sheet[name].write(j+1,1,float(var_mean.iloc[j,i]))
        sheet[name].write(j+1,0,datetime.fromtimestamp(time[j].timestamp()).strftime("%d")+datetime.fromtimestamp(time[j].timestamp()).strftime("%H"))
        for k in range(0,len(sta)):
            sheet[name].write(0,k+2,sta[k])
            sheet[name].write(j+1,k+2,sta_var[sta[k]].iloc[j,i])
f.save(os.path.abspath(os.path.join(path_output)+day+'.xls'))